# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [3]:
%matplotlib inline
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [4]:
# Importanto o Banco de Dados
data = pd.read_excel("tweets nescau classificação.xlsx")
data.head()

,Treinamento,Classificação
0,quero comer pão de coco c nescau,1
1,rt @tonystarkmeta: bolsonaro tomou uma facada ...,0
2,@iam_mudblood ah se vao liberar nem esquenta l...,0
3,28596 coisas pra estudar pra faculdade e pro t...,0
4,rt @gii_gcosta: fala uma verdade que niguem ac...,1


In [5]:
# Removendo caracteres irrelvantes.
def limpador(batata):
    batata0 = batata.str.replace("~", " ")
    batata1 = batata0.str.replace("@", " ")
    batata2 = batata1.str.replace(":"," ")
    batata3 = batata2.str.replace(","," ")
    batata4 = batata3.str.replace("."," ")
    batata5 = batata4.str.replace(";"," ")
    batata6 = batata5.str.replace("_"," ")
    batata7 = batata6.str.replace("-"," ")
    batata8 = batata7.str.replace("\n"," ")
    batata9 = batata8.str.replace(" a "," ")
    batata10 = batata9.str.replace(" que "," ")
    batata11 = batata10.str.replace(" o "," ")
    batata12 = batata11.str.replace("?"," ")
    batata13 = batata12.str.replace("!"," ")
    batata14 = batata13.str.replace("#"," ")
    batata15 = batata14.str.replace("$"," ")
    batata16 = batata15.str.replace("&"," ")
    batata17 = batata16.str.replace("*"," ")
    batata18 = batata17.str.replace(" se "," ")
    batata19 = batata18.str.replace("+"," ")
    batata20 = batata19.str.replace("rt "," ")
    batata21 = batata20.str.replace("="," ")
    batata22 = batata21.str.replace(">"," ")
    batata23 = batata22.str.replace("<"," ")
    batata24 = batata23.str.replace("^"," ")
    batata25 = batata24.str.replace("("," ")
    batata26 = batata25.str.replace(")"," ")
    batata27 = batata26.str.replace("2018"," ")
    batata28 = batata27.str.replace("'"," ")


# Transformando todas as palavras em minúsculas
    batata29 = batata28.str.lower()

#Removendo os espaços em branco e separando as palavras
    batata30 = batata29.str.split()
    return batata30


In [6]:
# Removendo caracteres irrelvantes.

limpador(data.Treinamento).head()

0             [quero, comer, pão, de, coco, c, nescau]
1    [tonystarkmeta, bolsonaro, tomou, uma, facada,...
2    [iam, mudblood, ah, vao, liberar, nem, esquent...
3    [28596, coisas, pra, estudar, pra, faculdade, ...
4    [gii, gcosta, fala, uma, verdade, niguem, acei...
Name: Treinamento, dtype: object

In [7]:
# Transformando em DataFrame
df_join = data.copy()
df_join["Treinamento"] = limpador(data["Treinamento"])
#df1 = pd.DataFrame(data=limpador(data.Treinamento))
#df1.head()

In [8]:
# Transformando em DataFrame
#df2 = pd.DataFrame(data=data["Classificação"])
#df2.head()

In [9]:
# Juntando ambos DataFrames em um único DataFrame mais limpo
#df_join = df1.join(df2, how="inner")
#df_join.head()

In [10]:
# Categorizando as classificações em Relevante( == 1) e Irrelevante( == 0)
df_join.Classificação = df_join.Classificação.astype("category")
df_join.Classificação.cat.categories = ("Irrelevante", "Relevante")
df_join.head()

,Treinamento,Classificação
0,"[quero, comer, pão, de, coco, c, nescau]",Relevante
1,"[tonystarkmeta, bolsonaro, tomou, uma, facada,...",Irrelevante
2,"[iam, mudblood, ah, vao, liberar, nem, esquent...",Irrelevante
3,"[28596, coisas, pra, estudar, pra, faculdade, ...",Irrelevante
4,"[gii, gcosta, fala, uma, verdade, niguem, acei...",Relevante


In [11]:
# Separando os DataFrames de acordo com a Categoria

df_relevante = df_join[df_join.Classificação=="Relevante"]
df_irrelevante = df_join[df_join.Classificação=="Irrelevante"]

df_relevante.head()


,Treinamento,Classificação
0,"[quero, comer, pão, de, coco, c, nescau]",Relevante
4,"[gii, gcosta, fala, uma, verdade, niguem, acei...",Relevante
5,"[bbhmymb, acabei, de, tomar, toddy, e, realmen...",Relevante
7,"[fsox7, nãoconfioemgenteque, acha, toddy, é, m...",Relevante
8,"[amo, nescau, deixar, bebo, toda, hora]",Relevante


In [12]:
# Porcentagem de Tweets

porcentagem_relevante = 100*(len(df_relevante) / len(df_join.Classificação))
porcentagem_irrelevante = 100*(len(df_irrelevante) / len(df_join.Classificação))

print("{:.2f}% de Tweets Relevantes".format(porcentagem_relevante))
print("{:.2f}% de Tweets Irrelevantes".format(porcentagem_irrelevante))


50.33% de Tweets Relevantes
49.67% de Tweets Irrelevantes


In [13]:
def classificador_naive(frase):
    
    serie = pd.Series(frase)
    serie_limpa = limpador(serie)
    df_serie_limpa = pd.DataFrame(data=serie_limpa)
    
    probabilidade_relevante = 1
    probabilidade_irrelevante = 1
    
    total_palavras=0
    
    for linha in df_join.Treinamento:
        for palavra in linha:
            total_palavras+=1
    

    for linha1 in df_serie_limpa[0]:
        for palavra1 in linha1:
            total_palavras_relevante = 0
            conta_palavra1_relevante = 0

            for linha in df_relevante.Treinamento:
                for palavra2 in linha:
                    total_palavras_relevante += 1
                    if palavra1==palavra2:
                        conta_palavra1_relevante += 1

            probabilidade_palavra1_relevante = (conta_palavra1_relevante+1)/(total_palavras_relevante+total_palavras) #Laplace

        probabilidade_relevante = probabilidade_relevante*probabilidade_palavra1_relevante
    
    for linha1 in df_serie_limpa[0]:
        for palavra1 in linha1:
            conta_palavra1_irrelevante = 0
            total_palavras_irrelevante = 0
            
            for linha in df_irrelevante.Treinamento:
                for palavra2 in linha:
                    total_palavras_irrelevante += 1
                    if palavra1==palavra2:
                        conta_palavra1_irrelevante += 1

            probabilidade_palavra1_irrelevante = (conta_palavra1_irrelevante+1)/(total_palavras_irrelevante+total_palavras) #Laplace

        probabilidade_irrelevante = probabilidade_irrelevante*probabilidade_palavra1_irrelevante

    if probabilidade_relevante>probabilidade_irrelevante:
        return("Relevante")
    else:
        return("Irrelevante")

classificador_naive( "para limpar bom é bolsonaro")

'Irrelevante'

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [14]:
def verificador_total(data):
    
    base_teste = pd.read_excel(data)
    
    df_teste = pd.DataFrame(data=base_teste)
    
    df_teste.Classificação = df_teste.Classificação.astype("category")
    df_teste.Classificação.cat.categories = ("Irrelevante", "Relevante")
    
    total_certo=0
    total_errado=0
    total=0
    
    positivos_falsos=0
    positivos_verdadeiros=0
    negativos_verdadeiros=0
    negativos_falsos=0
    
    i=0
    
    while i <len(df_teste):
        resultado=classificador_naive(df_teste["Teste"][i])
        
        if resultado==df_teste["Classificação"][i]:
            #print("acertou")
            if resultado=="Relevante":
                positivos_verdadeiros+=1 #Relevante e acertou
            else:
                negativos_verdadeiros+=1 #Irrelevante e acertou
            total_certo+=1
        else:
            #print("errou")
            if resultado=="Irrelevante":
                positivos_falsos+=1 #Relevante e errou
            else:
                negativos_falsos+=1 #Irrelevante e errou
            total_errado+=1
            
        total+=1
        
        i+=1
        
    probabilidade_certo=(total_certo/total)*100
    probabilidade_errado=100*total_errado/total
    
    prob_positivos_falsos=100*positivos_falsos/total
    prob_positivos_verdadeiros=100*positivos_verdadeiros/total
    prob_negativos_falsos=100*negativos_falsos/total
    prob_negativos_verdadeiros=100*negativos_verdadeiros/total
    
    print("Probabilidade de positivos falsos: {}%".format(prob_positivos_falsos))
    print("Probabilidade de positivos verdadeiros: {}%".format(prob_positivos_verdadeiros))
    print("Probabilidade de negativos falsos: {}%".format(prob_negativos_falsos))
    print("Probabilidade de negativos verdadeiros: {}%".format(prob_negativos_verdadeiros))
    
    
    
    
    return ("{}% de acerto".format(probabilidade_certo), "{}% de erro".format(probabilidade_errado))
    
    
verificador_total("tweets nescau classificação verificação.xlsx")



Probabilidade de positivos falsos: 7.5%
Probabilidade de positivos verdadeiros: 53.0%
Probabilidade de negativos falsos: 18.0%
Probabilidade de negativos verdadeiros: 21.5%


('74.5% de acerto', '25.5% de erro')

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


In [15]:
base_teste = pd.read_excel("tweets nescau classificação verificação.xlsx")

total=0
total_classificado_como_relevante=0
total_classificado_como_irrelevante=0

for i in base_teste.Classificação:
    total+=1
    if i==1:
        total_classificado_como_relevante+=1
    else:
        total_classificado_como_irrelevante+=1
        
pcntg_relevante=100*total_classificado_como_relevante/total
pcntg_irrelevante=100*total_classificado_como_irrelevante/total

print("{}% dos tweets foram classificados como relevantes na base teste".format(pcntg_relevante))
print("{}% dos tweets foram classificados como irrrelevantes na base teste".format(pcntg_irrelevante))


60.5% dos tweets foram classificados como relevantes na base teste
39.5% dos tweets foram classificados como irrrelevantes na base teste


  Cada vez mais, empresas necessitam de informações para avaliar o seu rendimento. Com isso, a busca por dados e postagens são algo de extremo interesse no mundo corporativo. Tendo isso em vista, as redes sociais desempenham um papel ideal de divulgação de informações, através de postagens de seus usuários, bastante relevantes para grandes corporações. Com o intuito de captar informações relevantes para empresas, foi criado um algoritmo que digere os dados, retornando apenas aqueles que possuem conteúdo significativo para a empresa. Assim, com o código acima, é possível simular um exemplo de Machine Learning, algo utilizado em grande escala, por empresas mundo a fora, para lidar com uma enorme quantidade de dados. Como funciona: inserimos uma série de conteúdos diversos no computador, e apartir desses conteúdos adicionados (servem como exemplo sobre o que é relevante e o que é irrelevante) o computador "aprende" a executar a tarefa de filtrar apenas conteúdo relevante para uma empresa. Tendo em vista a automação como fator diferencial no mercado, é possível expandir esse código de modo que este possa ser ainda mais preciso. Visando obter o melhor rendimento possível e conquistar a clientela, ganhando mais espaço no mercado, é imprescindível que investimentos sejam feitos nesse setor. 
  
  No modelo acima, foi utilizado tweets relacionados ao produto Nescau como exemplo. A partir dos resultados fornecidos pelo algoritmo podemos constatar que: na base teste 60.5% dos tweets que classificamos manualmente foram designados como relevantes, se simplesmente nossa função classificasse todos os tweets como relevantes teríamos como probabilidade de acerto de nosso classificador naive 60.5%. Contudo, como o objetivo é retornar o conteúdo relevante de maneira autônoma, o algoritmo foi implementado e obteve um graude de acerto de 74.5%. Isso mostra que usando o teorema de bayes conseguimos classificar corretamente 74.5% de tweets publicados por um público aleatório. Isso nos diz que a cada 4 tweets, 3 são classificados corretamente, um resultado incrível para um código que exerce uma função de maneira autônoma sem o auxílio de um ser humano. Além do código fornecer se o tweet é irrelevante ou relevante, também é retornado a porcentagem de tweets relevantes classificados corretamente e incorretamente, e a quantidade de tweets irrelevantes classificados erroneamente e de maneira correta.
   
   Uma das maneiras de melhorar o grau de acerto da nossa função, é colocando as probabilidades numa escala logarítmica que trataria com maior precisão as probabilidades dos tweets serem ou não relevantes. Traduzir as abreviações para seus respectivos significados, traduzir o significado de um emoji com relevante ou irrelevante, e buscar outras informações e postagens, que possam ser relevantes, dos usuários são algumas melhoras que podem acarretar em uma maior precisão do código para empresas.
   
   As mensagens de duplo sentido acabam por ser tratadas como irrelevantes, visto que o código não consegue destinguir através das palavras. Com isso, para que essas mensagens possam ser avaliadas de maneira correta, é necessário uma maior interpretação da frase, analisando o sentido de emojis publicados (se houverem alguns), analisar algumas "#" adicionadas e também outros tweets do mesmo usuário.
   
   Não podemos alimentar a base de Treinamento utilizando o próprio classificador, visto que trata-se de um exemplo de Machine Learning. A máquina "aprende" o que é relevante e irrelevante a partir de uma série de exemplos já classificados. Assim, através do calculo de probabilidades desses exemplos contendo inúmeras palavras, é possível determinar se uma frase é relevante ou não. Portanto, precisamos de um banco de dados classificados por um humano, que sirvam como exemplo para o código exercer sua função de maneira correta. Desse modo, não podemos utilizar o código nesse banco de dados, visto que dará um resultado 100% correto, algo que não será repetido para outros bancos de dados.
   
   Diferentes cenários para a utilização do Naive-Baies são: na medicina para diagonsticar uma pesso através de imagens, que possam apontar indícios de uma doença. Calcula-se a probabilidade de certa característica específica na imagem ser ou não um sinal, a partir de diversas amostras já classificadas, e apartir delas decidir se a imagem analisada possui ou não indícios de uma doença; no universo dos esportes, para calcular quais são as melhores condições para a realização do esporte, e obtenção dos melhores resultados. Isso é calculado através de informações prévias já fornecidas, por atletas que as obteram através experiências empíricas. Com essas informações é possível calcular se uma determinada condição será positiva ou negativa para a realização do esporte, ou se aquela condição influenciará de modo a acarretar resultados melhores ou piores do que a média. Além desses dois exemplos, uma última aplicação seria no Mercado Financeiro de modo a prever se a bolsa de valores terá uma tendência a melhorar ou não. Através de resultados anteriores, é possível prever se uma certa ação terá seu preço ascendente ou descendente em meio a uma sequência de ações que foram registradas e absorvidas pela máquina. Assim pode ser calculado a probabilidade da ação subir, de modo a maximizar os investimentos de um trader. 
   
   
   